In [6]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import datetime as datetime
import matplotlib.colors as mcolors
import seaborn as sns

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import interpolate


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
hemi = 'NH'
id = '1'

In [10]:
#ds = xr.open_dataset(f'/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT_Data/test/half_day_1deg.nc')
ds = xr.open_dataset(f'/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT_Data/multi/2020-2024_fac_feats_180_40_2min.nc')
#ds = ds.sel(dt=slice('2023-01-01', '2024-12-31'))
ds

<xarray.Dataset> Size: 26GB
Dimensions:     (dt: 916734, lat: 40, lon: 180)
Coordinates:
  * dt          (dt) datetime64[ns] 7MB 2020-12-29 ... 2024-07-31T23:58:00
  * lon         (lon) int32 720B 1 3 5 7 9 11 13 ... 347 349 351 353 355 357 359
  * lat         (lat) float64 320B 50.0 51.03 52.05 53.08 ... 87.95 88.97 90.0
Data variables:
    fac         (dt, lat, lon) float32 26GB ...
    np          (dt) float64 7MB ...
    vsw         (dt) float64 7MB ...
    By          (dt) float64 7MB ...
    Bz          (dt) float64 7MB ...
    tilt_angle  (dt) float64 7MB ...

In [12]:
def nan_count(ds, var):
    #count number of NaNs in a data variable
    con_data = ds[var].values
    nan_count = np.count_nonzero(np.isnan(con_data))
    nan_ratio = nan_count / con_data.size
    not_nan = con_data.size - nan_count
    return not_nan, nan_count, np.round(nan_ratio, 2)

nan_count(ds, 'np')

(916734, 0, 0.0)

In [7]:
def clock_angle(By, Bz):
    magnitude = np.sqrt(By**2 + Bz**2)
    cos_theta = np.arccos(Bz / magnitude)
    return cos_theta

def dayside_reconnection_rate(v_sw, By, Bz):
    By = By * 1e-9 #nT to T
    Bz = Bz * 1e-9
    Byz = np.sqrt(By**2 + Bz**2)
    v_sw = v_sw * 1e3 #km/s to m/s
    l_0 = 6371 * 1000 * 3.8 #3.8 R_E i m

    L_eff = l_0 * (v_sw/(4*10**5))**(1/3)
    theta = clock_angle(By, Bz)
    Phi_D = L_eff * v_sw * Byz * np.sin(theta/2)**(9/2)
    return Phi_D / 1e3

#ds['Phi_D'] = dayside_reconnection_rate(ds['flow_speed'], ds['BY_GSE'], ds['BZ_GSE'])
#ds

In [13]:
# Initialize scalers for the target variable and input variables
input_scaler = StandardScaler() #scale using standard dev where mean is 0 and std is 1

# Extract the target variable and reshape for scaling
target_var = ds['fac'].values  # shape (t (n), mlat (50), mlt (24))

# Extract and scale input variables (variables that are dependent only on 'dt')
input_vars = ['By', 'Bz', 'vsw', 'np', 'tilt_angle']
#input_vars = ['BY_GSE', 'BZ_GSE', 'flow_speed', 'proton_density', 'tilt_angle']
input_data = np.array([ds[var].values for var in input_vars]).T  # shape (22320, number_of_vars)
input_data_scaled = input_scaler.fit_transform(input_data)

file_path = f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_scaler_{hemi}_id{id}.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(input_scaler, file)

def create_sequences(target_data, input_data, lb):
    X, y = [], []
    for i in range(len(target_data) - lb):
        X.append(input_data[i:i+lb].T)
        y.append(target_data[i+lb])

    return np.array(X), np.array(y)

lookback = 30 # number of time steps to look back. At 1 min set to 60, at 2 min set to 30, and so on
X, y = create_sequences(target_var, input_data_scaled, lookback)

X.shape, y.shape

((916704, 5, 30), (916704, 40, 180))

In [14]:
def split_data(X, y, train_size=0.8, test_size=0.1):
    n = len(X)
    train_size = int(n*train_size)
    test_size = int(n*test_size)
    
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:train_size+test_size], y[train_size:train_size+test_size]
    X_val, y_val = X[train_size+test_size:], y[train_size+test_size:]
    
    return X_train, y_train, X_test, y_test, X_val, y_val

X_train, y_train, X_test, y_test, X_val, y_val = split_data(X, y)
X_train.shape, X_val.shape, X_test.shape

((733363, 5, 30), (91671, 5, 30), (91670, 5, 30))

In [15]:
# Convert data to PyTorch tensors and move to GPU
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = np.array(y_test)

# Create DataLoader
batch_size = 128
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [16]:
mlat_len = np.size(ds['lat'].values)
mlt_len = np.size(ds['lon'].values)

In [17]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, mlat_len * mlt_len)

    def forward(self, x):
        batch_size = x.size(0)
        # Use the same device as the model for hidden states
        device = next(self.parameters()).device  # Ensure hidden states are on the same device as the model
        
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size, device=device)  # Initial hidden state
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size, device=device)  # Initial cell state
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = out.view(-1, mlat_len, mlt_len)
        return out

    def reset_states(self):
        # Reset the internal states of the LSTM layer
        self.lstm.reset_parameters()

# Instantiate the model with lookback size
model = LSTM(lookback, 128, 2).to(device)
model

LSTM(
  (lstm): LSTM(30, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=7200, bias=True)
)

In [18]:
#calculate the area for a given latitude and longitude
def calculate_delta(lat1, lon1, lat2, lon2):
    lat1 = np.abs(lat1)
    lat2 = np.abs(lat2)
    
    radius = 6371.008 * 1000

    area = radius**2 * (np.sin(np.radians(lat2)) - np.sin(np.radians(lat1))) * (np.radians(lon2) - np.radians(lon1))
    
    return area 

#set up the latitude and longitude intervals
mlat = np.linspace(40,90,mlat_len)
mlt = np.linspace(0,360,mlt_len) 

#calculate the area for each latitude and longitude interval
def calculate_area(mlat, mlt):
    areas = np.zeros((mlat_len-1, mlt_len-1))
    for i in range(len(mlat) - 1):
        for j in range(len(mlt) - 1):
            lat1, lat2 = mlat[i], mlat[i + 1]
            lon1, lon2 = mlt[j], mlt[j + 1]

            area = calculate_delta(lat1, lon1, lat2, lon2)
            areas[i, j] = area

    #interpolate the areas back to shape of mlat, mlt
    #Currently 49x23, interpolate back to 50x24
    x = np.arange(areas.shape[1])
    y = np.arange(areas.shape[0])
    f = interpolate.interp2d(x, y, areas, kind='linear')
    xnew = np.arange(0, areas.shape[1], areas.shape[1]/(areas.shape[1]+1))
    ynew = np.arange(0, areas.shape[0], areas.shape[0]/(areas.shape[0]+1))
    areas = f(xnew, ynew)

    return areas

areas = calculate_area(mlat, mlt)

def weighted_loss_function(y_true, y_pred):

    # Normalize the area
    weights = areas / np.min(areas)

    #create custom 'Mean Area Weighted Loss' (MAWE) function
    loss = torch.mean(torch.abs(y_true - y_pred) * torch.tensor(weights, dtype=torch.float32).to(device))
    
    return loss

/tmp/ipykernel_9715/1215014759.py:31: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f = interpolate.interp2d(x, y, areas, kind='linear')
/tmp/ipykernel_9715/1215014759.py:34: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` 

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=7e-4) # Adam optimizer
scheduler = StepLR(optimizer, step_size=8, gamma=0.7) #decay learning rate by 0.7 every 8 epochs

In [20]:
# Model settings
num_epochs = 50
train_losses = []
val_losses = []

# Early stopping
best_val_loss = float('inf')
patience = 7
counter = 0

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader:
        # Move data to the appropriate device
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        train_outputs = model(X_batch)
        # loss = loss_function(train_outputs, y_batch)
        loss = weighted_loss_function(y_batch, train_outputs)

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update model parameters
        epoch_loss += loss.item()  # Accumulate epoch loss

    # Step the scheduler
    scheduler.step()  # Adjust learning rate

    train_loss = epoch_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            # Move data to the appropriate device
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            val_outputs = model(X_batch)
            # val_loss += loss_function(val_outputs, y_batch).item()
            val_loss += weighted_loss_function(y_batch, val_outputs).item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_model_{hemi}_id{id}.pt')
    else:
        counter += 1
        if counter == patience:
            print(f'Validation loss did not improve for {patience} epochs. Stopping training.')
            break

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss (MAWE): {train_loss:.4f}, Val Loss (MAWE): {val_loss:.4f}, Best Val Loss: {best_val_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.1e}')


Epoch [1/50], Train Loss (MAWE): 1.3785, Val Loss (MAWE): 1.4584, Best Val Loss: 1.4584, LR: 7.0e-04
Epoch [2/50], Train Loss (MAWE): 1.2415, Val Loss (MAWE): 1.3998, Best Val Loss: 1.3998, LR: 7.0e-04
Epoch [3/50], Train Loss (MAWE): 1.2136, Val Loss (MAWE): 1.3944, Best Val Loss: 1.3944, LR: 7.0e-04
Epoch [4/50], Train Loss (MAWE): 1.1998, Val Loss (MAWE): 1.3862, Best Val Loss: 1.3862, LR: 7.0e-04
Epoch [5/50], Train Loss (MAWE): 1.1906, Val Loss (MAWE): 1.3835, Best Val Loss: 1.3835, LR: 7.0e-04
Epoch [6/50], Train Loss (MAWE): 1.1826, Val Loss (MAWE): 1.3800, Best Val Loss: 1.3800, LR: 7.0e-04
Epoch [7/50], Train Loss (MAWE): 1.1762, Val Loss (MAWE): 1.3796, Best Val Loss: 1.3796, LR: 7.0e-04
Epoch [8/50], Train Loss (MAWE): 1.1705, Val Loss (MAWE): 1.3795, Best Val Loss: 1.3795, LR: 4.9e-04
Epoch [9/50], Train Loss (MAWE): 1.1592, Val Loss (MAWE): 1.3835, Best Val Loss: 1.3795, LR: 4.9e-04
Epoch [10/50], Train Loss (MAWE): 1.1542, Val Loss (MAWE): 1.3771, Best Val Loss: 1.3771, L

In [21]:
test_model = LSTM(lookback, 128, 2)
test_model.load_state_dict(torch.load(f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_model_{hemi}_id{id}.pt'))
test_model.to('cpu')
test_model.eval()

# List to store predictions
predictions = []

# Perform predictions in batches
with torch.no_grad():
    batch_size = 128
    for i in range(0, len(X_test), batch_size):
        X_batch = X_test[i:i+batch_size]
        
        # Forward pass
        y_pred = test_model(X_batch)
        
        # Append the predictions (already on CPU)
        predictions.append(y_pred.numpy())

# Concatenate all predictions into a single array
predictions = np.concatenate(predictions)

In [23]:
def batch_mae(y_test, predictions, batch_size):
    mae_accum = []
    
    for i in range(0, len(y_test), batch_size):
        y_batch = y_test[i:i+batch_size]
        pred_batch = predictions[i:i+batch_size]
        mae_accum.append(np.mean(np.abs(y_batch - pred_batch)))

    return np.mean(mae_accum)

def batch_nmae_std(y_test, predictions, batch_size):
    overall_std = np.std(y_test)  # Use the overall standard deviation
    nmae_accum = []

    for i in range(0, len(y_test), batch_size):
        y_batch = y_test[i:i+batch_size]
        pred_batch = predictions[i:i+batch_size]
        nmae_accum.append(np.mean(np.abs(y_batch - pred_batch)) / overall_std)

    return np.mean(nmae_accum)

def batch_corrcoef(y_test, predictions, batch_size):
    corr_accum = []
    
    for i in range(0, len(y_test), batch_size):
        y_batch = y_test[i:i+batch_size]
        pred_batch = predictions[i:i+batch_size]
        if len(y_batch) > 1:  # Corrcoef needs at least 2 points to calculate
            corr_accum.append(np.corrcoef(y_batch.flatten(), pred_batch.flatten())[0, 1])
    
    return np.mean(corr_accum)

def skill(m, o):
    skill = 1 - (np.sum((m - o)**2) / np.sum((o - np.mean(o))**2))
    return skill      

# Set the batch size
batch_size = 10000

# Calculate metrics in batches
MAE = batch_mae(y_test, predictions, batch_size)
NMAE_std = batch_nmae_std(y_test, predictions, batch_size)  # Now using overall std of y_test
R = batch_corrcoef(y_test, predictions, batch_size)

# Calculate skill normally (if this isn't memory intensive)
PE = skill(predictions, y_test)

print(f'Test Loss (MAE): {MAE:.4f}, Test Loss (NMAE_std): {NMAE_std:.4f}, R: {R:.2f}, PE: {PE:.4f}')
#print(f'Test Loss (MAE): {MAE:.4f}, Test Loss (NMAE_std): {NMAE_std:.4f}')


Test Loss (MAE): 0.0370, Test Loss (NMAE_std): 0.4451, R: 0.63, PE: 0.3879


: 

Test Loss (MAE): 0.0370, Test Loss (NMAE_std): 0.4451, R: 0.63, PE: 0.3879